In [45]:
#import requests
import polars as pl
from fyers_apiv3 import fyersModel
import json
import datetime as dt
from dateutil.relativedelta import relativedelta
import pyotp
import time
import pickle
import pyperclip
import webbrowser
from urllib.parse import parse_qs,urlparse
import os
import pandas as pd
import altair as alt

fd=pl.read_parquet(f'C:\\Users\\{os.getlogin()}\\OneDrive\\Python\\stockdata.parquet')
with open(f"C:\\Users\\{os.getlogin()}\\Downloads\\fyers.json") as f:
    f=json.load(f) 

sym="TCS"

In [46]:
import talib as tl
def indicators(df,sym):
    df=df.filter(pl.col('symbol')==sym)
    df=df.with_columns(SMA_200=tl.SMA(df['close'],timeperiod=200))
    df=df.with_columns(stockh_k=tl.STOCHRSI(df['close'],timeperiod=10)[0],stockh_d=tl.STOCHRSI(df['close'],timeperiod=10)[1])
    return df
#indicators(fd,sym)
fd=fd.rename({'open':'Open','high':'High','low':'Low','close':'Close','volume':'Volume'})

In [47]:

fd1=fd.filter(pl.col('symbol')==sym)
fd1=fd1.with_columns(month=pl.col('epoch').dt.month())
fd1=fd1.with_columns(day=pl.col('epoch').dt.weekday())
alt.Chart(fd1).mark_line().encode(x='epoch:T',y='Close:Q',tooltip=['Close:Q','epoch:T']).properties(width=800)

alt.Chart(...)

In [48]:
bb=alt.Chart(fd1).encode(x='day:O',y='Close:Q').properties(height=150)
(bb.mark_boxplot().encode(tooltip=['Close:Q','epoch:T']).properties(width=420) & bb.transform_density(
    'Close',as_=['Close', 'density'],groupby=['day']).mark_area(orient='horizontal').encode(x=alt.X('density:Q').stack('center').impute(None).title(None).axis(
    labels=False, values=[0], grid=False, ticks=True),y='Close:Q',color='day:N',xOffset='day:N',column=alt.Column('day:N').spacing(0)
        .header(titleOrient='bottom', labelOrient='bottom', labelPadding=0)).properties(width=60)).configure_view(stroke=None)

alt.VConcatChart(...)

In [49]:
fd2=fd1.sort('epoch')[['epoch','Close','month','day']]
fd2=fd2.with_columns(lag2=pl.col('Close').shift(2))

In [50]:
import xgboost as xgb
reg=xgb.XGBRegressor()
reg.fit(fd2.drop(['epoch','Close'])[:-12].to_pandas(),fd2['Close'][:-12].to_pandas())
pred=reg.predict(fd2.drop(['epoch','Close'])[-12:].to_pandas())
pp=pl.DataFrame([fd2['epoch'][-12:],pl.Series(pred)])

In [51]:
brush = alt.selection_interval(encodings=['x'])
lower = alt.Chart(fd1).mark_line().encode(x='epoch:T',y='Close:Q',tooltip=['Close:Q','epoch:T']).properties(height=60).add_params(brush)
(alt.Chart(fd1).mark_line().encode(x=alt.X('epoch:T').scale(domain=brush),y='Close:Q',tooltip=['Close:Q','epoch:T']).properties(width=800) + alt.Chart(pp).mark_line().encode(x=
                                                                alt.X('epoch:T').scale(domain=brush), y='column_1:Q',tooltip=['column_1:Q','epoch:T']).properties(width=800)) & lower

alt.VConcatChart(...)

In [15]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import numpy as np
from datetime import date

class SMAcross(Strategy):
    n1 = 30
    n2 = 90
    def init(self):
        price = self.data['Close']
        self.sma1 = tl.SMA(df['Close'],timeperiod=30)
        self.sma2 = tl.SMA(df['Close'],timeperiod=60)
    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()
df=fd.filter(pl.col('symbol')==sym).filter(pl.col('epoch')>=date(2021,9,9)).drop('symbol').sort('epoch',descending=False)
df=df.to_pandas()
df['epoch']=pd.to_datetime(df['epoch'])
#df=df.with_columns(pl.col('epoch').cast(pl.String))
bt = Backtest(df.set_index('epoch'), SMAcross,cash= 100000,exclusive_orders=True)
stats = bt.run()
bt.plot()

C:\Users\smishra14\setup\miniconda\envs\algo\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p2575', ...)

In [1]:
bt.run()

NameError: name 'bt' is not defined

In [ ]:
def generate_access_token() :
    global user_id, pin, app_id, api_key, redirect_url, totp_key, url
    print("\nGenerating Access Token .....................") 
    try :
        user_id = f['ci']
        pin = f['pin']
        app_id = f['client_id']
        api_key = f['secret_key']
        redirect_url = f['redirect_uri']
        totp_key = f['totp']
        session = fyersModel.SessionModel(
        client_id = app_id,
        secret_key = api_key,
        redirect_uri = redirect_url,
        response_type = "code",
        state=f['state'])
        url = session.generate_authcode()
        webbrowser.open_new(url)
        old_clipboard_contents = pyperclip.paste()
        new_clipboard_contents = pyperclip.paste()
        while old_clipboard_contents == new_clipboard_contents : 
            new_clipboard_contents = pyperclip.paste()
            time.sleep(1)
        url = new_clipboard_contents
        parsed = urlparse(url)
        auth_code = parse_qs(parsed.query)["auth_code"][0]
        session = fyersModel.SessionModel(client_id = app_id,secret_key = api_key,redirect_uri = redirect_url,response_type = "code",grant_type = "authorization_code")
        session.set_token(auth_code)
        response = session.generate_token()
        access_token = response["access_token"]
        refresh_token = response["refresh_token"]
        token_dict = {"app_id": app_id,"access_token": access_token,"refresh_token": refresh_token}
        with open ("token_dict.pickle", "wb") as file :
            pickle.dump(token_dict, file)
    except :
        time.sleep(2)
        generate_access_token()
def fyers_login() :
    global fyers, token_dict, name
    while True:
        try:
            with open("token_dict.pickle", "rb") as file :
                token_dict = pickle.load(file)
                #print(token_dict)
        except :
            token_dict = {"app_id": 0, "access_token": 0, "refresh_token": 0}
        fyers = fyersModel.FyersModel(client_id=token_dict["app_id"], is_async=False, token= token_dict["access_token"], log_path='')
        response = fyers.get_profile()
        if response["s"] == "error" :
            generate_access_token()
        else :
            print("\nlogin Details ..........")
            #print(name)
            break
fyers_login()

In [23]:
ny=5
try:
    print('try')
    sd=fd.filter(pl.col('symbol')==sym)['epoch'].max().strftime('%Y-%m-%d')
    ed=(dt.datetime.now()-dt.timedelta(hours=24)).strftime('%Y-%m-%d')
    std=fyers.history(data={'symbol':f'NSE:{sym}-EQ','resolution':'1D','date_format':1,'range_from':sd,'range_to':ed,'cont_flag':1})
    df=pl.DataFrame(std['candles'],schema={'epoch':pl.Int32,'open':pl.Float32,'high':pl.Float32,'low':pl.Float32,'close':pl.Float32,'volume':pl.Float32},orient="row")
    df=df.with_columns(epoch=pl.from_epoch('epoch'))
    df=df.with_columns(symbol=pl.lit(sym))
    fd=pl.concat([fd,df])
    fd.write_parquet('C:\\Users\\smishra14\\OneDrive\\Python\\stockdata.parquet')
except:
    print('except')
    for i in range(ny):
        print('loop')
        ed=(dt.datetime.now()-dt.timedelta(hours=24)-relativedelta(years=i)).strftime('%Y-%m-%d')
        sd=(dt.datetime.now()-dt.timedelta(hours=24)-relativedelta(years=i+1)).strftime('%Y-%m-%d')
        std=fyers.history(data={'symbol':f'NSE:{sym}-EQ','resolution':'1D','date_format':1,'range_from':sd,'range_to':ed,'cont_flag':1})
        df=pl.DataFrame(std['candles'],schema={'epoch':pl.Int32,'open':pl.Float32,'high':pl.Float32,'low':pl.Float32,'close':pl.Float32,'volume':pl.Float32},orient="row")
        df=df.with_columns(epoch=pl.from_epoch('epoch'))
        df=df.with_columns(symbol=pl.lit(sym))
        fd=pl.concat([fd,df])
    fd.write_parquet('C:\\Users\\smishra14\\OneDrive\\Python\\stockdata.parquet')

try
except
loop
loop
loop
loop
loop


In [ ]:
import json
import pyotp
with open("C:\\Users\\mswas\\Downloads\\fyers.json") as f:
    f=json.load(f)
pyotp.TOTP(f['totp']).now()

In [2]:
from pyecharts.charts import Bar, Candlestick, Kline
from pyecharts import options as opts
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

df=fd.filter(pl.col('symbol')==sym).sort('epoch',descending=False)
df=df.with_columns(gain=pl.when(pl.col('open')<pl.col('close')).then(1).otherwise(-1))
bar = (
    Candlestick(init_opts=opts.InitOpts(height="600px",width="1320px"))
    .add_xaxis(df['epoch'].cast(pl.String).to_list())
    .add_yaxis(series_name="", y_axis=df['open','close','low','high'].to_numpy().tolist(),)
    #.add_dataset(data=list(df['open','high','low','close','gain'].to_numpy().tolist()),)
    .set_global_opts(title_opts=opts.TitleOpts(title=f"{sym}"), datazoom_opts=[
                opts.DataZoomOpts(is_show=False,type_="inside",xaxis_index=[0, 1],range_start=90,range_end=100,),
        opts.DataZoomOpts(is_show=True,xaxis_index=[0, 5],type_="slider",pos_bottom="5%",range_start=85,range_end=100,)],
                    tooltip_opts=opts.TooltipOpts(trigger="axis",axis_pointer_type="cross",background_color="rgba(245, 245, 245, 0.8)",border_width=1,border_color="#ccc",
                                                  textstyle_opts=opts.TextStyleOpts(color="#000"),),)
)
bar.load_javascript()
bar.render_notebook()

In [20]:
import altair as alt
source = fd.filter(symbol=sym)
brush = alt.selection_interval(encodings=['x'])
nearest = alt.selection_point(nearest=True, on="pointerover",fields=["epoch"], empty=False)
rules = alt.Chart(source).mark_rule(color="gray").encode(x="epoch:T",).transform_filter(nearest)
open_close_color = alt.condition("datum.open <= datum.close",alt.value("#06982d"),alt.value("#ae1325"))
base = alt.Chart(source).encode(alt.X('epoch:T').scale(domain=brush).axis(format='%m/%d', labelAngle=-45).title('Date'),tooltip=['open','high','low','close'],
                                color=open_close_color).transform_filter(brush).add_params(nearest)
rule = base.mark_rule().encode(alt.Y('low:Q').title('Price').scale(zero=False),alt.Y2('high:Q'))
bar = base.mark_bar().encode(alt.Y('open:Q'),alt.Y2('close:Q'))
lower =  alt.Chart(source).mark_area(opacity=.6).encode(x = 'epoch:T',y = 'volume:Q').properties(height=75,width=1200).add_params(brush)

((rule + bar + rules).properties(width=1200,height=400) & lower)

alt.VConcatChart(...)

In [ ]:
import shutil
shutil.copyfile('C:\\Users\\smishra14\\OneDrive - Stryker\\python\\algo.ipynb', 'C:\\Users\\smishra14\\OneDrive\\Python\\notebooks\\algo.ipynb')

In [ ]:
df=df.sort('epoch').with_columns(pl.col('close').rolling_mean(3,by='epoch').alias('cl 3ma'))
df.with_columns(pl.col('close')-pl.col('cl 3ma'))